##### 데이터 준비 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

#### 랜덤포레스트 

* 부트스트랩 샘플 

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
# 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


결정 트리에서의 특성 중요도보다 극값(?)이 완화되었다.

이는 특성의 일부를 랜덤하게 선택해 훈련하였기 때문에, 하나의 특성에 과도하게 집중되지 않고, 여러 특성이 훈련할 기회를 줌

=> 과대적합 줄이고, 일반화 가능성 높임 

In [7]:
# OOB 샘플 : 부트스트랩 샘플에 포함되지 않고 남는 샘풀 (검증 세트의 역할)
# OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리 평가
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수 (scores['test_score'])와 비슷하다. 

OOB 점수를 사용하면 교차검증 대신할 수 있어 훈련세트에 더 많은 샘플 사용 가능하다.

#### 엑스트라 트리 

* 전체 훈련 세트 사용, 노드 분할 시 무작위 분할 

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [10]:
# 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#### 그레이디언트 부스팅

* 깊이가 얕은 결정트리를 점점 추가하며 가장 낮은 곳을 찾아 이동(경사 하강)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합 되지 않음 (깊이가 얕은 결정트리이기에)

In [13]:
# 결정 트리개수 (100->) 500개, 학습률 (0.1->) 0.2
gb = GradientBoostingClassifier(random_state = 42, n_estimators = 500, learning_rate = 0.2)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [15]:
# 특성중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그레이디언트 부스팅은 랜덤포레스트보다 일부 특성에 더 집중한다.

#### 히스토그램 기반 그레이디언트 부스팅 

* 입력특성 256개 구간 

In [16]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [17]:
# 특성 중요도 : 특성을 하나씩 랜덤하게 섞어 모델 성능이 변화하는지 관찰
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [19]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [20]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [21]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)  #  히스토그램 기반 그레디언트 부스팅
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [23]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
